In [1]:
from pathlib import Path

from ase import units, Atoms
from ase.build import molecule
from ase.io import read, write
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect import flow
from prefect_dask import DaskTaskRunner
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

from mlip_arena.models.utils import REGISTRY, MLIPEnum
from mlip_arena.tasks.run import md as MD

In [ ]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [ ]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [ ]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [ ]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f'{atoms.get_chemical_formula()}.extxyz', atoms)

In [2]:
atoms = read("H256O128.extxyz")
print(atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


In [3]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

cluster_kwargs = dict(
    cores=1,
    memory="64 GB",
    shebang="#!/bin/bash",
    account="matgen",
    walltime="02:00:00",
    job_mem="0",
    job_script_prologue=[
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    job_directives_skip=["-n", "--cpus-per-task", "-J"],
    job_extra_directives=[
        "-J combustion-water",
        "-q regular",
        f"-N {nodes_per_alloc}",
        "-C gpu",
        f"-G {gpus_per_alloc}",
        f"--exclusive",
        # "--time-min=00:30:00",
        # "--comment=1-00:00:00",
        # "--signal=B:USR1@60",
        # "--requeue",
        # "--open-mode=append"
    ],
    death_timeout=86400
)


cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())
cluster.adapt(minimum_jobs=10, maximum_jobs=11)
client = Client(cluster)

#!/bin/bash

#SBATCH -A matgen
#SBATCH --mem=0
#SBATCH -t 02:00:00
#SBATCH -J combustion-water
#SBATCH -q regular
#SBATCH -N 1
#SBATCH -C gpu
#SBATCH -G 4
#SBATCH --exclusive
source ~/.bashrc
module load python
source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.18:43951 --name dummy-name --nthreads 1 --memory-limit 59.60GiB --nanny --death-timeout 86400



2024-09-27 03:54:04,812 - distributed.scheduler - ERROR - Task md-2-9b52cfa3e22647de99659385eca3d3f2-1 marked as failed because 4 workers died while trying to run it
2024-09-27 03:54:04,820 - distributed.scheduler - ERROR - Task md-5-2a33088343d64b66b612e4145ffaf870-1 marked as failed because 4 workers died while trying to run it
2024-09-27 03:54:04,824 - distributed.scheduler - ERROR - Task md-0-017eef8ab8744d7c958badc5d53fdab4-1 marked as failed because 4 workers died while trying to run it
2024-09-27 05:55:56,468 - distributed.scheduler - ERROR - Task md-8-bcd5e0bb82854952b2c5c430d4c2e7e8-1 marked as failed because 4 workers died while trying to run it
2024-09-27 05:55:56,906 - distributed.scheduler - ERROR - Task md-6-a575c0cc3b1e4fc984560ff101dbfc6c-1 marked as failed because 4 workers died while trying to run it
2024-09-27 05:55:56,907 - distributed.scheduler - ERROR - Task md-9-f98b1f7c61c749c8b67cd669517a49bd-1 marked as failed because 4 workers died while trying to run it
2024

In [4]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        future = MD.submit(
            atoms=atoms,
            calculator_name=model,
            calculator_kwargs=None,
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            ase_md_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            mb_velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)
    
    return [future.result() for future in futures]

In [5]:
results = combustion(atoms)

23:19:58.587 | INFO    | prefect.engine - Created flow run 'tangerine-mussel' for flow 'combustion'

23:19:58.592 | INFO    | Flow run 'tangerine-mussel' - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/flow-runs/flow-run/ea339a90-929f-491d-bf23-e257c925ad6b

23:19:58.593 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://128.55.64.18:43951

23:19:58.599 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://128.55.64.18:8787/status

23:19:59.454 | INFO    | Flow run 'tangerine-mussel' - Created task run 'md-2' for task 'md'

...

05:57:04.333 | INFO    | Task run 'md-7' - Crash detected! Execution was interrupted by an unexpected exception: KilledWorker: Attempted to run task 'md-7-40b7b4ede41745e29d5c5fd0cc4d9e42-1' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.68.6:39739. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

05:57:04.976 | INFO    | Task run 'md-8' - Crash detected! Execution was interrupted by an unexpected exception: KilledWorker: Attempted to run task 'md-8-bcd5e0bb82854952b2c5c430d4c2e7e8-1' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.66.238:37065. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

05:57:05.435 | INFO    | Task run 'md-9' - Crash detected! Execution was interrupted by an unexpected exception: KilledWorker: Attempted to run task 'md-9-f98b1f7c61c749c8b67cd669517a49bd-1' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.65.221:34843. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

05:57:05.840 | ERROR   | Flow run 'tangerine-mussel' - Finished in state Failed("Flow run encountered an exception. KilledWorker: Attempted to run task 'md-2-9b52cfa3e22647de99659385eca3d3f2-1' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.66.32:43067. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.")

KilledWorker: Attempted to run task 'md-2-9b52cfa3e22647de99659385eca3d3f2-1' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.66.32:43067. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.